## SICP 习题 （2.40）解题总结：unique-pairs

SICP习题2.40要求我们实现一个叫unique-pairs的函数，用于生成小于等于n的所有整数组合。

然后题目要求我们使用(unique-pairs)函数去简化书中提到的(prime-sum-pairs)函数

首先我们需要面对的是如何生成小于等于某一数的所有数字的序列，书中提到了(enumerate-interval)函数，函数定义如下：

In [1]:
(define (enumerate-interval low high) 
   (if (> low high) 
       '()
       (cons low (enumerate-interval (+ low 1) high)))) 

像下面这样的调用就可以生成0到10的序列

In [2]:
(enumerate-interval 0 10)

(0 1 2 3 4 5 6 7 8 9 10)

接着可以通过以下的(map)函数生成i和比i小的数的所有序对：

In [3]:
(define i 10)
(map
  (lambda (j) 
    (list i j))
  (enumerate-interval 1 (- i 1)))

((10 1) (10 2) (10 3) (10 4) (10 5) (10 6) (10 7) (10 8) (10 9))

在外面再套一层map函数就可以生成比n小的所有序对了：

In [4]:
(define n 10)
(map (lambda (i) 
	 (map
	  (lambda (j) 
	    (list i j))
	  (enumerate-interval 1 (- i 1))))
       (enumerate-interval 1 n))

(() ((2 1)) ((3 1) (3 2)) ((4 1) (4 2) (4 3)) ((5 1) (5 2) (5 3) (5 4)) ((6 1) (6 2) (6 3) (6 4) (6 5)) ((7 1) (7 2) (7 3) (7 4) (7 5) (7 6)) ((8 1) (8 2) (8 3) (8 4) (8 5) (8 6) (8 7)) ((9 1) (9 2) (9 3) (9 4) (9 5) (9 6) (9 7) (9 8)) ((10 1) (10 2) (10 3) (10 4) (10 5) (10 6) (10 7) (10 8) (10 9)))

这个差不多已经是题目要求的结果了，就是格式不太对。

题目要求我们把生成的序对放在一个序列里，我们这里生成的是嵌套的序列。

通过accumulate把他们串起来就好了：

In [5]:
(define (accumulate op initial sequence)
  (if (null? sequence)
      initial
      (op (car sequence)
	  (accumulate op initial (cdr sequence)))))



In [6]:
(define (unique-pairs n)
  (accumulate append '() (map (lambda (i) 
	 (map
	  (lambda (j) 
	    (list i j))
	  (enumerate-interval 1 (- i 1))))
       (enumerate-interval 1 n))))

In [7]:
(unique-pairs 10)

((2 1) (3 1) (3 2) (4 1) (4 2) (4 3) (5 1) (5 2) (5 3) (5 4) (6 1) (6 2) (6 3) (6 4) (6 5) (7 1) (7 2) (7 3) (7 4) (7 5) (7 6) (8 1) (8 2) (8 3) (8 4) (8 5) (8 6) (8 7) (9 1) (9 2) (9 3) (9 4) (9 5) (9 6) (9 7) (9 8) (10 1) (10 2) (10 3) (10 4) (10 5) (10 6) (10 7) (10 8) (10 9))

然后就是去简化(prime-sum-pairs)函数， 先看看检查一个序对和是不是质数的方法：

先拷贝之前的检查质数的prime?函数：

In [8]:

(define (smallest-divisor n)
  (find-divisor n 2))

(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
	((divides? test-divisor n) test-divisor)
	(else (find-divisor n (+ test-divisor 1)))))

(define (divides? a b)
  (= (remainder b a) 0))

(define (square x)
  (* x x))

(define (prime? n)
  (= n (smallest-divisor n)))

检查序对和是不是质数的方法定义如下：

In [9]:
(define (prime-sum? pair)
  (prime? (+ (car pair) (cadr pair))))

通过map调用prime-sum?就可以计算哪组序对满足条件了：

In [10]:
(map prime-sum? (unique-pairs 10))

(#t #f #t #t #f #t #f #t #f #f #t #f #f #f #t #f #f #f #t #f #t #f #f #t #f #t #f #f #f #t #f #t #f #f #f #t #t #f #t #f #f #f #t #f #t)

接着想用系统的filter函数来直接过滤出来，后来发现 Calysto Scheme的filter实现有点奇怪，老是报错，于是就自己实现了一遍：

In [11]:
(define (filter function input-list)
  (if (pair? input-list)
      (if (function (car input-list))
          (cons (car input-list) (filter function (cdr input-list)))
          (filter function (cdr input-list)))
      ()
      ))

效果如下：

In [12]:
(filter  prime-sum? (unique-pairs n))

((2 1) (3 2) (4 1) (4 3) (5 2) (6 1) (6 5) (7 4) (7 6) (8 3) (8 5) (9 2) (9 4) (9 8) (10 1) (10 3) (10 7) (10 9))

最后就是按(prime-sum-paris)的要求把序对的和也算出来放进序列里，定义成函数如下：

In [13]:
(define (prime-sum-pairs n)
  (map (lambda (pair-sum)
	 (append pair-sum (list (+ (car pair-sum) (cadr pair-sum)))))

   (filter  (lambda (pair) 
	      (prime-sum? pair)) (unique-pairs n))))

执行效果如下：

In [18]:
(prime-sum-pairs 10)

((2 1 3) (3 2 5) (4 1 5) (4 3 7) (5 2 7) (6 1 7) (6 5 11) (7 4 11) (7 6 13) (8 3 11) (8 5 13) (9 2 11) (9 4 13) (9 8 17) (10 1 11) (10 3 13) (10 7 17) (10 9 19))